# Import Libraries

In [1]:
import numpy as np 
import pandas as pd
import zipfile
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from keras.optimizers import SGD
from sklearn.preprocessing import StandardScaler
import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Unzip the Directory

In [ ]:
def unzip_dir(parent_dir):
  # Loop through each folder in the parent directory
  for foldername in os.listdir(parent_dir):
    folderpath = os.path.join(parent_dir)

    # Check if the folder is a directory
    if os.path.isdir(folderpath):
      # Loop through each file in the folder
      for filename in os.listdir(parent_dir):
        filepath = os.path.join(parent_dir, filename)

      # Check if the file is a zip file
      if filename.endswith(".zip"):
              
        # Create a ZipFile object for the file
        with zipfile.ZipFile(filepath, "r") as zip_ref:
          # Extract all the files to the parent_dir
          zip_files=zip_ref.extractall(parent_dir)
          print(zip_files)
    else:
      print("No Dir")

In [ ]:
parent_dir = "/content/mydrive/MyDrive/NS-vehicle-anomaly-dataset-mega/ConstSpeed_0709/"
extract_zip_file=unzip_dir(parent_dir)
print(extract_zip_file)

# Convert Dataset Json to CSV Formate

In [ ]:
def read_json(json_directory):    

    # Get a list of all the JSON files in the directory
    json_files = [f for f in os.listdir(json_directory) if f.endswith(".json")]

    # Initialize an empty list to store the dataframes
    dataframes = []

    # Loop through each JSON file and read it into a Pandas dataframe
    for json_file in json_files:
        # Construct the full path to the JSON file
        json_file_path = os.path.join(json_directory, json_file)
        
        # Read the JSON file into a dataframe
        dataframe = pd.read_json(json_file_path, lines=True)
        
        # Append the dataframe to the list of dataframes
        dataframes.append(dataframe)

    # Concatenate all the dataframes into a single dataframe
    merged_dataframe = pd.concat(dataframes, ignore_index=True)
    # Save the combine dataframe
    merged_dataframe.to_csv('/content/ConstSpeed_0709.csv')
    return merged_dataframe

In [ ]:
json_directory='/content/mydrive/MyDrive/NS-vehicle-anomaly-dataset-mega/ConstSpeed_0709/VeReMi_28800_32400_2022-9-11_20:46:34'
meraged_dataframe=read_json(json_directory)
print(meraged_dataframe)

# Combine ALL CSV Files Formate(14-Attackers)

In [ ]:
# set the directory where the CSV files are stored
directory = '/content/mydrive/MyDrive/CSV-files-14-Attacks/'

# create an empty list to store the dataframes
df_list = []

# loop through each file in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        # read the first 10000 rows from the CSV file into a dataframe
        df = pd.read_csv(os.path.join(directory, filename), nrows=10000)
        # append the dataframe to the list
        df_list.append(df)

# concatenate all the dataframes into a single dataframe
combined_df = pd.concat(df_list)
 # position column 
combined_df['pos_0'] = combined_df['pos'].str.strip('[]').str.split(', ').str[0].astype(float)
combined_df['pos_1'] = combined_df['pos'].str.strip('[]').str.split(', ').str[1].astype(float)
combined_df['pos_2'] = combined_df['pos'].str.strip('[]').str.split(', ').str[2].astype(float)
combined_df.drop(columns=['pos'], inplace=True)


# speed column 
combined_df['spd_0'] = combined_df['spd'].str.strip('[]').str.split(', ').str[0].astype(float)
combined_df['spd_1'] = combined_df['spd'].str.strip('[]').str.split(', ').str[1].astype(float)
combined_df['spd_2'] = combined_df['spd'].str.strip('[]').str.split(', ').str[2].astype(float)
combined_df.drop(columns=['spd'], inplace=True)

# print the first few rows of the combined dataframe
print(combined_df.head())

### Select Specific Columns From Dataframe

In [ ]:
# split the dataframe into pos , type , spd and attacks
combined_df=combined_df[['type', 'pos_0', 'pos_1', 'spd_0', 'spd_1', 'Attacks']]
# combine csv file
combined_df.to_csv('/content/train_dataset.csv')

# Load Dataset

In [2]:
# load dataset
df=pd.read_csv('/kaggle/input/vermie-dataset/train_dataset.csv')
print(df)

        type       pos_0        pos_1      spd_0      spd_1        Attacks
0          0  143.932447  1022.904829   0.000000   0.000000       ConstPos
1          0  354.420706   641.637176  -2.108532  16.227503       ConstPos
2          0  153.586183   900.726664   0.000149   0.000149       ConstPos
3          0  928.797437  1091.099487   0.000020   0.000020       ConstPos
4          0  155.782562   771.833062  -1.968970  13.744119       ConstPos
...      ...         ...          ...        ...        ...            ...
159995     1  132.119616   889.446690   0.000505   0.000505  DoSDisruptive
159996     1  510.372108   860.120894   8.855281   8.277780  DoSDisruptive
159997     1  630.371460   420.831258  14.423676   5.560019  DoSDisruptive
159998     1  800.305077   791.123899  12.422521  -3.610628  DoSDisruptive
159999     1  838.941172   742.790481  11.638469  -9.348271  DoSDisruptive

[160000 rows x 6 columns]


### Data Preprocessing

In [ ]:
# identify the null values
df.isnull().sum()

# Step 1 To Find the Type of Attacks(0-1)

# Loading Dataset

In [3]:
combined_df_type=pd.read_csv('/kaggle/input/vermie-dataset/train_dataset.csv')
# First Five Rows
print(combined_df_type.head(5))

   type       pos_0        pos_1     spd_0      spd_1   Attacks
0     0  143.932447  1022.904829  0.000000   0.000000  ConstPos
1     0  354.420706   641.637176 -2.108532  16.227503  ConstPos
2     0  153.586183   900.726664  0.000149   0.000149  ConstPos
3     0  928.797437  1091.099487  0.000020   0.000020  ConstPos
4     0  155.782562   771.833062 -1.968970  13.744119  ConstPos


## Feature Engineering

In [5]:
# Split the data into features (X) and target (y)
X = combined_df_type.drop(['type', 'Attacks'], axis=1)
y = combined_df_type['type']
# Split the data into training and test sets
X_train_type, X_test_type, y_train_type, y_test_type = train_test_split(X, y, test_size=0.2)
print(X_train_type.shape)
print(X_test_type.shape)
print(y_train_type.shape)
print(y_test_type.shape)

(128000, 4)
(32000, 4)
(128000,)
(32000,)


# Test Case Shuffle Dataset- Type as a Target

In [41]:
pred_df=combined_df_type
# Seperate the 0-non attacker and 1-attacker from original dataframe
pred_df_type_0=pred_df.loc[pred_df['type']==0]
pred_df_type_1=pred_df.loc[pred_df['type']==1]
# List
final_dataset=[]
# Non Attacker -0 slice into 20 rows
combine_pred_df_type_0=pred_df_type_0.loc[:20]
# append into the list
final_dataset.append(combine_pred_df_type_0)
pred_df_type_1=pred_df.loc[pred_df['type']==1]
pred_df_type_1=pred_df_type_1.reset_index(drop=True)
# Attacker-1 slice into 10 rows
combine_pred_df_type_1=pred_df_type_1.loc[:10]
# append into list
final_dataset.append(combine_pred_df_type_1)

In [42]:
# concatinate the list
final_train_data=pd.concat(final_dataset)
# Drop the indexs
final_train_data=final_train_data.reset_index(drop=True)
# shuffle the dataframe random
shuffled_df = final_train_data.sample(frac = 1)
# reindexs the shuffle dataframe
shuffled_df = shuffled_df.reset_index()
# drop the index columns
shuffled_df = shuffled_df.drop('index', axis=1)

In [ ]:
df = pd.DataFrame(shuffled_df) # your original dataframe

# create list to hold the new dataframes
dfs = []
# loop through 6 times, each time increasing the number of rows
for i in range(1, 7):
    rows = i * 20 # calculate number of rows
    new_df = df.iloc[:rows]
     # slice the dataframe to get the desired number of rows
    dfs.append(new_df) # add the new dataframe to the list

# Training Machine Learning Models -Type as a Target


### Test Case

In [ ]:
from sklearn.metrics import classification_report
def test_cases(model, dfs):
    model_accuracy = []
    final_times = []
    precision_scores=[]
    recall_scores=[]
    f1_scores=[]
    for test in dfs:
        # splitting features and target
        X = test.drop(['type', 'Attacks'], axis=1)
        y = test['type']
        
        start_time = time.time()
        #  prediction
        y_pred = model.predict(X)
        # end time
        end_time = time.time()
        accuracy = accuracy_score(y, y_pred)
        
        # precision tp / (tp + fp)
        precision= precision_score(y, y_pred,average='weighted')
        # print(precision)
        # recall: tp / (tp + fn)
        recall = recall_score(y, y_pred)
        # f1: 2 tp / (2 tp + fp + fn)
        f1 = f1_score(y, y_pred)
        final_time = end_time - start_time
        model_accuracy.append(accuracy)
        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)
        final_times.append(final_time)

    return model_accuracy, final_times, precision_scores, recall_scores, f1_scores


# Decision Tree Classification

In [ ]:
dcc= tree.DecisionTreeClassifier(criterion="log_loss",max_depth=50,min_samples_leaf=3,max_features=4)
dcc_model_step_1=dcc.fit(X_train_type , y_train_type)
y_pred_type = dcc_model_step_1.predict(X_test_type)
decision_tree = y_pred_type
accuracy = accuracy_score(y_test_type, y_pred_type)
print("Accuracy:", accuracy)

# Test Case 1



> Accuracy of test case on Five Test Messages



In [ ]:
mod_acc = test_cases(dcc_model_step_1, dfs)
# display in table
final_test_step_1_1={ '5msg' : mod_acc[0][0] , '10msg' : mod_acc[0][1] , '15msg' : mod_acc[0][2], '20msg' : mod_acc[0][3] , '25msg' : mod_acc[0][4] , '30msg' : mod_acc[0][5]}
Final_Time_step_1_1={ '5msg' : mod_acc[1][0] , '10msg' : mod_acc[1][1] , '15msg' : mod_acc[1][2], '20msg' : mod_acc[1][3] , '25msg' : mod_acc[1][4] , '30msg' : mod_acc[1][5]}
test_1_dataframe_step_1=pd.DataFrame([final_test_step_1_1 , Final_Time_step_1_1] , index=['DT' , 'Time'])
print(test_1_dataframe_step_1)



> Compare Accuracy Vs Time



In [ ]:
sns.barplot(test_1_dataframe_step_1)
plt.xlabel('DTC')
plt.ylabel('Time')
plt.title('Prediction of DTC Vs Time')

# Evaluation

In [ ]:
mod_acc

In [ ]:
# display in table
# display in table
Accurcay_step_1_1={ '5msg' : mod_acc[0][0] , '10msg' : mod_acc[0][1] , '15msg' : mod_acc[0][2], '20msg' : mod_acc[0][3] , '25msg' : mod_acc[0][4] , '30msg' : mod_acc[0][5]}
Preciouse_step_1_1={ '5msg' : mod_acc[2][0] , '10msg' : mod_acc[2][1] , '15msg' : mod_acc[2][2], '20msg' : mod_acc[2][3] , '25msg' : mod_acc[2][4] , '30msg' : mod_acc[2][5]}
Recall_step_1_1={ '5msg' : mod_acc[3][0] , '10msg' : mod_acc[3][1] , '15msg' : mod_acc[3][2], '20msg' : mod_acc[3][3] , '25msg' : mod_acc[3][4] , '30msg' : mod_acc[3][5]}
f1_step_1_1={ '5msg' : mod_acc[4][0] , '10msg' : mod_acc[4][1] , '15msg' : mod_acc[4][2], '20msg' : mod_acc[4][3] , '25msg' : mod_acc[4][4] , '30msg' : mod_acc[4][5]}


EVALU_1_dataframe_step_1=pd.DataFrame([Accurcay_step_1_1 , Preciouse_step_1_1 , Recall_step_1_1 , f1_step_1_1 ] , index=['Accurcay' , 'Preciouse'  , ' Recall', 'f1_score'])
print(EVALU_1_dataframe_step_1)

# KNN Model

In [ ]:
neigh_STEP_1 = KNeighborsClassifier(n_neighbors=3, weights= "distance", leaf_size=50, n_jobs=-1)
neigh_STEP_1.fit(X_train_type , y_train_type)
y_pred = neigh_STEP_1.predict(X_test_type)
knn_model = y_pred
prob_acc = neigh_STEP_1.predict_proba(X_test_type)
accuracy = accuracy_score(y_test_type, y_pred)
print("Accuracy:", accuracy)

# Test Case 2



> Accuracy of test case on Five Test Messages



In [ ]:
mod_acc = test_cases(neigh_STEP_1, dfs)
# display in table
final_test_step_1_2={ '5msg' : mod_acc[0][0] , '10msg' : mod_acc[0][1] , '15msg' : mod_acc[0][2], '20msg' : mod_acc[0][3] , '25msg' : mod_acc[0][4] , '30msg' : mod_acc[0][5]}
Final_Time_step_1_2={ '5msg' : mod_acc[1][0] , '10msg' : mod_acc[1][1] , '15msg' : mod_acc[1][2], '20msg' : mod_acc[1][3] , '25msg' : mod_acc[1][4] , '30msg' : mod_acc[1][5]}
test_2_dataframe_step_1=pd.DataFrame([final_test_step_1_2 , Final_Time_step_1_2] , index=['KNN' , 'Time'])




> Compare the KNN Vs Time



In [ ]:
sns.barplot(test_2_dataframe_step_1)
plt.xlabel('KNN')
plt.ylabel('Time')
plt.title('Prediction of KNN Vs Time')
print(test_2_dataframe_step_1)

# Evaluation

In [ ]:
mod_acc

In [ ]:
# display in table
Accurcay_step_1_2={ '5msg' : mod_acc[0][0] , '10msg' : mod_acc[0][1] , '15msg' : mod_acc[0][2], '20msg' : mod_acc[0][3] , '25msg' : mod_acc[0][4] , '30msg' : mod_acc[0][5]}
Preciouse_step_1_2={ '5msg' : mod_acc[2][0] , '10msg' : mod_acc[2][1] , '15msg' : mod_acc[2][2], '20msg' : mod_acc[2][3] , '25msg' : mod_acc[2][4] , '30msg' : mod_acc[2][5]}
Recall_step_1_2={ '5msg' : mod_acc[3][0] , '10msg' : mod_acc[3][1] , '15msg' : mod_acc[3][2], '20msg' : mod_acc[3][3] , '25msg' : mod_acc[3][4] , '30msg' : mod_acc[3][5]}
f1_step_1_2={ '5msg' : mod_acc[4][0] , '10msg' : mod_acc[4][1] , '15msg' : mod_acc[4][2], '20msg' : mod_acc[4][3] , '25msg' : mod_acc[4][4] , '30msg' : mod_acc[4][5]}

EVALU_2_dataframe_step_1=pd.DataFrame([Accurcay_step_1_2 , Preciouse_step_1_2 , Recall_step_1_2 , f1_step_1_2 ] , index=['Accurcay' , 'Preciouse'  , ' Recall' , 'F1_Score'])
print(EVALU_2_dataframe_step_1)

# Random Forest Model

In [ ]:
rfc_step_1 = RandomForestClassifier(criterion="entropy",class_weight='balanced')
rfc_step_1.fit(X_train_type, y_train_type)
y_pred = rfc_step_1.predict(X_test_type)
rf_model = y_pred
accuracy = accuracy_score(y_test_type, y_pred)
print("Accuracy:", accuracy)

# Test Case 3



> Accuracy of test case on Five Test Messages



In [ ]:
mod_acc = test_cases(rfc_step_1, dfs)
# display in table
final_test_step_1_3={ '5msg' : mod_acc[0][0] , '10msg' : mod_acc[0][1] , '15msg' : mod_acc[0][2], '20msg' : mod_acc[0][3] , '25msg' : mod_acc[0][4] , '30msg' : mod_acc[0][5]}
Final_Time_step_1_3={ '5msg' : mod_acc[1][0] , '10msg' : mod_acc[1][1] , '15msg' : mod_acc[1][2], '20msg' : mod_acc[1][3] , '25msg' : mod_acc[1][4] , '30msg' : mod_acc[1][5]}
test_3_dataframe_step_1=pd.DataFrame([final_test_step_1_3 , Final_Time_step_1_3] , index=['RF' , 'Time'])



> Compare the RFC Vs Time



In [ ]:
sns.barplot(test_3_dataframe_step_1)
plt.xlabel('RF')
plt.ylabel('Time')
plt.title('Prediction of RF Vs Time')
print(test_3_dataframe_step_1)

# Evaluation

In [ ]:
# display in table
Accurcay_step_1_3={ '5msg' : mod_acc[0][0] , '10msg' : mod_acc[0][1] , '15msg' : mod_acc[0][2], '20msg' : mod_acc[0][3] , '25msg' : mod_acc[0][4] , '30msg' : mod_acc[0][5]}
Preciouse_step_1_3={ '5msg' : mod_acc[2][0] , '10msg' : mod_acc[2][1] , '15msg' : mod_acc[2][2], '20msg' : mod_acc[2][3] , '25msg' : mod_acc[2][4] , '30msg' : mod_acc[2][5]}
Recall_step_1_3={ '5msg' : mod_acc[3][0] , '10msg' : mod_acc[3][1] , '15msg' : mod_acc[3][2], '20msg' : mod_acc[3][3] , '25msg' : mod_acc[3][4] , '30msg' : mod_acc[3][5]}
f1_step_1_3={ '5msg' : mod_acc[4][0] , '10msg' : mod_acc[4][1] , '15msg' : mod_acc[4][2], '20msg' : mod_acc[4][3] , '25msg' : mod_acc[4][4] , '30msg' : mod_acc[4][5]}


EVALU_3_dataframe_step_1=pd.DataFrame([Accurcay_step_1_3 , Preciouse_step_1_3 , Recall_step_1_3 , f1_step_1_3 ] , index=['Accurcay' , 'Preciouse'  , ' Recall' , 'F1_Score'])
print(EVALU_3_dataframe_step_1)

# LSTM

In [ ]:
# convert dataframes to numpy arrays
X_train = np.array(X_train_type)
y_train = np.array(y_train_type)
X_test = np.array(X_test_type)
y_test = np.array(y_test_type)

# reshape input data
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
# define the model
LSTM_model_step_1 = Sequential()
LSTM_model_step_1.add(LSTM(100, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
LSTM_model_step_1.add(Dropout(0.5))
LSTM_model_step_1.add(LSTM(100, return_sequences=True))
LSTM_model_step_1.add(Dropout(0.2))
LSTM_model_step_1.add(LSTM(100))
LSTM_model_step_1.add(Dropout(0.2))
LSTM_model_step_1.add(Dense(1, activation='relu'))
LSTM_model_step_1.compile(optimizer='adam', loss='mse', metrics=['accuracy'])


# fit the model
LSTM_model_step_1.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

In [ ]:
# evaluate the model
mse = LSTM_model_step_1.evaluate(X_test, y_test, verbose=1)
score_step_1 = LSTM_model_step_1.evaluate(X_test, y_test,batch_size = 10)

In [ ]:
LSTM_model_step_1.fit(X_train, y_train, epochs=10, batch_size=32)

In [ ]:
def test_cases(model, dfs):
    model_accuracy = []
    final_times = []
    precision_scores=[]
    recall_scores=[]
    f1_scores=[]
    for test in dfs:
        # splitting features and target
        X = test.drop(['type', 'Attacks'], axis=1)
        y = test['type']
        
        start_time = time.time()
        #  prediction
        y_pred = model.predict(X)
        y_pred=np.where(y_pred > 0.5 , 0, 1)
        # end time
        end_time = time.time()
        accuracy = accuracy_score(y, y_pred)
        
        # precision tp / (tp + fp)
        precision= precision_score(y, y_pred)
        # print(precision)
        # recall: tp / (tp + fn)
        recall = recall_score(y, y_pred)
        # f1: 2 tp / (2 tp + fp + fn)
        f1 = f1_score(y, y_pred)
        final_time = end_time - start_time
        model_accuracy.append(accuracy)
        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)
        final_times.append(final_time)

    return model_accuracy, final_times, precision_scores, recall_scores, f1_scores


# Test Case 4 

In [ ]:
mod_acc = test_cases(LSTM_model_step_1, dfs)
# display in table
final_test_step_1_4={ '5msg' : mod_acc[0][0] , '10msg' : mod_acc[0][1] , '15msg' : mod_acc[0][2], '20msg' : mod_acc[0][3] , '25msg' : mod_acc[0][4] , '30msg' : mod_acc[0][5]}
Final_Time_step_1_4={ '5msg' : mod_acc[1][0] , '10msg' : mod_acc[1][1] , '15msg' : mod_acc[1][2], '20msg' : mod_acc[1][3] , '25msg' : mod_acc[1][4] , '30msg' : mod_acc[1][5]}
# test_4_dataframe_step_1=pd.DataFrame([final_test_step_1_4 , Final_Time_step_1_4] , index=['lstm' , 'Time'])
test_4_dataframe_step_1=pd.DataFrame([final_test_step_1_4, Final_Time_step_1_4] , index=['lstm' , 'Time'])

In [ ]:
sns.barplot(test_4_dataframe_step_1)
plt.xlabel('LSTM')
plt.ylabel('Time')
plt.title('Prediction of LSTM Vs Time')
print(test_4_dataframe_step_1)
plt.savefig('lstm_step_1.png')

# Evaluation

In [ ]:
mod_acc

In [ ]:
# display in table
Accurcay_step_1_4={ '5msg' : mod_acc[0][0] , '10msg' : mod_acc[0][1] , '15msg' : mod_acc[0][2], '20msg' : mod_acc[0][3] , '25msg' : mod_acc[0][4] , '30msg' : mod_acc[0][5]}
Preciouse_step_1_4={ '5msg' : mod_acc[2][0] , '10msg' : mod_acc[2][1] , '15msg' : mod_acc[2][2], '20msg' : mod_acc[2][3] , '25msg' : mod_acc[2][4] , '30msg' : mod_acc[2][5]}
Recall_step_1_4={ '5msg' : mod_acc[3][0] , '10msg' : mod_acc[3][1] , '15msg' : mod_acc[3][2], '20msg' : mod_acc[3][3] , '25msg' : mod_acc[3][4] , '30msg' : mod_acc[3][5]}
f1_step_1_4={ '5msg' : mod_acc[4][0] , '10msg' : mod_acc[4][1] , '15msg' : mod_acc[4][2], '20msg' : mod_acc[4][3] , '25msg' : mod_acc[4][4] , '30msg' : mod_acc[4][5]}

EVALU_4_dataframe_step_1=pd.DataFrame([Accurcay_step_1_4 , Preciouse_step_1_4 , Recall_step_1_4 , f1_step_1_4] , index=['Accurcay' , 'Preciouse'  , ' Recall' , 'F1_Score'])
print(EVALU_4_dataframe_step_1)

# GRU

In [ ]:
# The GRU architecture
regressorGRU_step_1 = Sequential()
# First GRU layer with Dropout regularisation
regressorGRU_step_1.add(GRU(units=50, return_sequences=True, input_shape=(X_train_type.shape[1],1), activation='tanh'))
regressorGRU_step_1.add(Dropout(0.2))
# Second GRU layer
regressorGRU_step_1.add(GRU(units=50, return_sequences=True, input_shape=(X_train_type.shape[1],1), activation='tanh'))
regressorGRU_step_1.add(Dropout(0.2))
# Third GRU layer
regressorGRU_step_1.add(GRU(units=50, return_sequences=True, input_shape=(X_train_type.shape[1],1), activation='tanh'))
regressorGRU_step_1.add(Dropout(0.2))
regressorGRU_step_1.add(GRU(units=50, activation='tanh'))
regressorGRU_step_1.add(Dropout(0.2))
regressorGRU_step_1.add(Dense(units=1))
regressorGRU_step_1.compile(optimizer=SGD(lr=0.01, decay=1e-7, momentum=0.9, nesterov=False),loss='mean_squared_error' , metrics=['accuracy'])
regressorGRU_step_1.fit(X_train_type,y_train_type,batch_size=150)
y_pred=regressorGRU_step_1.predict(X_test_type)

# Test Case 5

In [ ]:
mod_acc = test_cases(regressorGRU_step_1, dfs)
# display in table
final_test_step_1_5={ '5msg' : mod_acc[0][0] , '10msg' : mod_acc[0][1] , '15msg' : mod_acc[0][2], '20msg' : mod_acc[0][3] , '25msg' : mod_acc[0][4] , '30msg' : mod_acc[0][5]}
Final_Time_step_1_5={ '5msg' : mod_acc[1][0] , '10msg' : mod_acc[1][1] , '15msg' : mod_acc[1][2], '20msg' : mod_acc[1][3] , '25msg' : mod_acc[1][4] , '30msg' : mod_acc[1][5]}
test_5_dataframe_step_1=pd.DataFrame([final_test_step_1_5 , Final_Time_step_1_5] , index=['GRU' , 'Time'])
print(test_5_dataframe_step_1)
sns.barplot(test_5_dataframe_step_1)
plt.xlabel('GRU')
plt.ylabel('Time')
plt.title('test_5_dataframe_step_1')
plt.savefig('GRU_step_1.png')

# Evaluation

In [ ]:
# display in table
Accurcay_step_1_5={ '5msg' : mod_acc[0][0] , '10msg' : mod_acc[0][1] , '15msg' : mod_acc[0][2], '20msg' : mod_acc[0][3] , '25msg' : mod_acc[0][4] , '30msg' : mod_acc[0][5]}
Preciouse_step_1_5={ '5msg' : mod_acc[2][0] , '10msg' : mod_acc[2][1] , '15msg' : mod_acc[2][2], '20msg' : mod_acc[2][3] , '25msg' : mod_acc[2][4] , '30msg' : mod_acc[2][5]}
Recall_step_1_5={ '5msg' : mod_acc[3][0] , '10msg' : mod_acc[3][1] , '15msg' : mod_acc[3][2], '20msg' : mod_acc[3][3] , '25msg' : mod_acc[3][4] , '30msg' : mod_acc[3][5]}
f1_step_1_5={ '5msg' : mod_acc[4][0] , '10msg' : mod_acc[4][1] , '15msg' : mod_acc[4][2], '20msg' : mod_acc[4][3] , '25msg' : mod_acc[4][4] , '30msg' : mod_acc[4][5]}

EVALU_5_dataframe_step_1=pd.DataFrame([Accurcay_step_1_5 , Preciouse_step_1_5 , Recall_step_1_5 , f1_step_1_5] , index=['Accurcay' , 'Preciouse'  , ' Recall' , 'F1_Score'])
print(EVALU_5_dataframe_step_1)

In [ ]:
X_train = np.array(X_train_type)
y_train = np.array(y_train_type)
X_test = np.array(X_test_type)
y_test = np.array(y_test_type)

stacked_X_train = np.column_stack((decision_tree_predictions, knn_predictions))
stacked_X_test = np.column_stack((decision_tree, knn_model, rf_model,  ))

# Step 2  - Attacker as a Tareget Varaiable

In [ ]:
# load dataset
data = pd.read_csv('/kaggle/input/vermie-dataset/train_dataset.csv')
print(data)

In [ ]:
# select the specific columns
data1 = data[['pos_0','pos_0','spd_0','spd_1']]

### Scale the Dataset

In [ ]:
data1 = np.array(data1)
# Scale the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data1[:, 1:])

In [ ]:
data1

In [ ]:
x = scaled_data
y= data['type']

In [ ]:
x

In [ ]:
dcc= tree.DecisionTreeClassifier()
dcc_model_step_1=dcc.fit(x , y)
y_pred_type = dcc_model_step_1.predict(x)
accuracy = accuracy_score(y, y_pred_type)
print("Accuracy:", accuracy)

In [ ]:
Y = data["Attacks"]

In [ ]:
label = Y.unique()

In [ ]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(Y)

# DTC Model

In [ ]:
dtc_step_2_1= tree.DecisionTreeClassifier()
dcc_model_step_2_1=dtc_step_2_1.fit(x , encoded_labels)
y_pred = dcc_model_step_2_1.predict(x)
accuracy = accuracy_score(encoded_labels, y_pred)
print("Accuracy:", accuracy)

In [ ]:
data.shape

In [ ]:
# load the combine dataframe 
combined_df_attacks=pd.read_csv('/kaggle/input/vermie-dataset/train_dataset.csv')

In [ ]:
le=preprocessing.LabelEncoder()
combined_df_attacks['Attacks']=le.fit_transform(combined_df_attacks['Attacks'])

### Feature Engineering

In [ ]:
# Split the data into features (X) and target (y)
X_attacks = combined_df_attacks.drop(['type', 'Attacks'], axis=1)
X_attacks = np.array(X_attacks)
scaler = StandardScaler()
X_attacks = scaler.fit_transform(X_attacks[:, 1:])

label_encoder = LabelEncoder()
combined_df_attacks['Attacks'] = label_encoder.fit_transform(combined_df_attacks['Attacks'])
y_attacks = combined_df_attacks['Attacks']
# Split the data into training and test sets
X_train_attacks, X_test_attacks, y_train_attacks, y_test_attacks = train_test_split(X_attacks, y_attacks, test_size=0.2)
print(X_train_attacks.shape)
print(X_test_attacks.shape)
print(y_train_attacks.shape)
print(y_test_attacks.shape)

In [ ]:
pd.concat([pd.DataFrame(X_attacks), y_attacks], ignore_index=True,axis=1)

In [ ]:
shuffled_df_attacks = pd.concat([pd.DataFrame(X_attacks), y_attacks], ignore_index=True,axis=1).sample(frac = 1)
shuffled_df_attacks = shuffled_df_attacks.reset_index()
final_shuffled_df_attacks = shuffled_df_attacks.drop('index', axis=1)
final_shuffled_df_attacks

In [ ]:
df = pd.DataFrame(final_shuffled_df_attacks)

In [ ]:
df1=[]
for i in range(df.shape[0]):
    x= df.iloc[i].drop(3)
    x = np.array(x).reshape(1, -1)
    pred =  dcc_model_step_1.predict(x)
    if(pred):
        df1.append(df.iloc[i])

In [ ]:
df1 = pd.DataFrame(df1)

In [ ]:
dfs_attacks=[]
# loop through 6 times, each time increasing the number of rows
for i in range(25, 55,5):
    #  # slice the dataframe to get the desired number of rows
    dfs_attacks.append(df1.iloc[:i]) # add the new dataframe to the list

# Test Case

In [ ]:
def test_cases(model_step_1, model_step_2, dfs):
    model_accuracy = []
    Final_time=[]
    precision_scores=[]
    recall_scores=[]
    f1_scores=[]
    for test in dfs:
        # splitting features and target
        X = test.drop(columns=[3], axis=1)
        y_attacks = test[3]
        
        #    print(X.iloc[[row]])
        start_time=time.time()
        y_pred_step_2 = model_step_2.predict(X) 
        End_time=time.time()
        accuracy = accuracy_score(y_attacks, y_pred_step_2)
       
        
        final_time=End_time-start_time

        
               
            #    print(model_accuracy)
        Final_time.append(final_time)
        model_accuracy.append(accuracy)
        

    return model_accuracy , Final_time 

# Training Machine Learning Models

In [ ]:
dtc_step_2_1= tree.DecisionTreeClassifier()
dcc_model_step_2_1=dtc_step_2_1.fit(X_train_attacks , y_train_attacks)
y_pred = dcc_model_step_2_1.predict(X_test_attacks)
accuracy = accuracy_score(y_test_attacks, y_pred)
print("Accuracy:", accuracy)

### Test Case 1

In [ ]:
accuracy_model=test_cases(dcc_model_step_1 , dcc_model_step_2_1 , dfs_attacks)

In [ ]:
# display in table
final_test_step_1_5={ '25msg' : accuracy_model[0][0] , '30msg' : accuracy_model[0][1] , '35msg' : accuracy_model[0][2], '40msg' : accuracy_model[0][3] , '45msg' : accuracy_model[0][4] , '50msg' : accuracy_model[0][5]}
Final_Time_step_1_5={ '25msg' : accuracy_model[1][0] , '30msg' : accuracy_model[1][1] , '35msg' : accuracy_model[1][2], '40msg' : accuracy_model[1][3] , '45msg' : accuracy_model[1][4] , '50msg' : accuracy_model[1][5]}
test_1_dataframe_step_2=pd.DataFrame([final_test_step_1_5 , Final_Time_step_1_5] , index=['DT' , 'Time'])
print(test_1_dataframe_step_2)

# KNN 

# Step 1

In [ ]:
x = scaled_data
y= data['type']

In [ ]:
KNN=KNeighborsClassifier()
KNN_model_step_1=KNN.fit(x , y)
y_pred_type = KNN_model_step_1.predict(x)
accuracy = accuracy_score(y, y_pred_type)
print("Accuracy:", accuracy)

# Step 2

In [ ]:
df1=[]
for i in range(df.shape[0]):
    x= df.iloc[i].drop(3)
    x = np.array(x).reshape(1, -1)
    pred =  KNN_model_step_1.predict(x)
    if(pred):
        df1.append(df.iloc[i])

In [ ]:
KNN=KNeighborsClassifier()
KNN_model_step_2_1=KNN.fit(X_train_attacks , y_train_attacks)
y_pred = KNN_model_step_2_1.predict(X_test_attacks)
accuracy = accuracy_score(y_test_attacks, y_pred)
print("Accuracy:", accuracy)

# Test Case

In [ ]:
accuracy_model=test_cases(KNN_model_step_1 , KNN_model_step_2_1 , dfs_attacks)
# display in table
final_test_step_1_5={ '25msg' : accuracy_model[0][0] , '30msg' : accuracy_model[0][1] , '35msg' : accuracy_model[0][2], '40msg' : accuracy_model[0][3] , '45msg' : accuracy_model[0][4] , '50msg' : accuracy_model[0][5]}
Final_Time_step_1_5={ '25msg' : accuracy_model[1][0] , '30msg' : accuracy_model[1][1] , '35msg' : accuracy_model[1][2], '40msg' : accuracy_model[1][3] , '45msg' : accuracy_model[1][4] , '50msg' : accuracy_model[1][5]}
test_1_dataframe_step_2=pd.DataFrame([final_test_step_1_5 , Final_Time_step_1_5] , index=['KNN' , 'Time'])
print(test_1_dataframe_step_2)

# Random Forest Classifier

### Step 1

In [ ]:
x = scaled_data
y= data['type']
rf = RandomForestClassifier()
rf_model_step_1=rf.fit(x , y)
y_pred_type = rf_model_step_1.predict(x)
accuracy = accuracy_score(y, y_pred_type)
print("Accuracy:", accuracy)

### Step 2

In [ ]:
df1=[]
for i in range(df.shape[0]):
    x= df.iloc[i].drop(3)
    x = np.array(x).reshape(1, -1)
    pred =  rf_model_step_1.predict(x)
    if(pred):
        df1.append(df.iloc[i])

In [ ]:
rf = RandomForestClassifier()
rf_model_step_2_1=rf.fit(X_train_attacks , y_train_attacks)
y_pred = rf_model_step_2_1.predict(X_test_attacks)
accuracy = accuracy_score(y_test_attacks, y_pred)
print("Accuracy:", accuracy)

## Test Case 2

In [ ]:
accuracy_model=test_cases(rf_model_step_1 , rf_model_step_2_1 , dfs_attacks)
# display in table
final_test_step_1_5={ '25msg' : accuracy_model[0][0] , '30msg' : accuracy_model[0][1] , '35msg' : accuracy_model[0][2], '40msg' : accuracy_model[0][3] , '45msg' : accuracy_model[0][4] , '50msg' : accuracy_model[0][5]}
Final_Time_step_1_5={ '25msg' : accuracy_model[1][0] , '30msg' : accuracy_model[1][1] , '35msg' : accuracy_model[1][2], '40msg' : accuracy_model[1][3] , '45msg' : accuracy_model[1][4] , '50msg' : accuracy_model[1][5]}
test_1_dataframe_step_2=pd.DataFrame([final_test_step_1_5 , Final_Time_step_1_5] , index=['RF' , 'Time'])
print(test_1_dataframe_step_2)

# GRU

### Step 1

In [ ]:
x = scaled_data.reshape(-1, 3, 1)
y = data['type']

# Define the GRU model with the correct input shape
regressorGRU_step_1_1 = Sequential()
regressorGRU_step_1_1.add(GRU(units=50, return_sequences=True, input_shape=(3, 1), activation='tanh'))
regressorGRU_step_1_1.add(Dropout(0.2))
regressorGRU_step_1_1.add(GRU(units=50, return_sequences=True, activation='tanh'))
regressorGRU_step_1_1.add(Dropout(0.2))
regressorGRU_step_1_1.add(GRU(units=50, return_sequences=True, activation='tanh'))
regressorGRU_step_1_1.add(Dropout(0.2))
regressorGRU_step_1_1.add(GRU(units=50, activation='tanh'))
regressorGRU_step_1_1.add(Dropout(0.2))
regressorGRU_step_1_1.add(Dense(units=1))
regressorGRU_step_1_1.compile(optimizer=SGD(learning_rate=0.01, decay=1e-7, momentum=0.9, nesterov=False), loss='mean_squared_error', metrics=['accuracy'])
regressorGRU_step_1_1.fit(x, y, batch_size=150)
y_pred = regressorGRU_step_1_1.predict(x)

# Step 2

In [ ]:
x = scaled_data.reshape(-1, 3, 1)
y = data['type']

# Define the GRU model with the correct input shape
regressorGRU_step_1_1 = Sequential()
regressorGRU_step_1_1.add(GRU(units=50, return_sequences=True, input_shape=(3, 1), activation='tanh'))
regressorGRU_step_1_1.add(Dropout(0.2))
regressorGRU_step_1_1.add(GRU(units=50, return_sequences=True, activation='tanh'))
regressorGRU_step_1_1.add(Dropout(0.2))
regressorGRU_step_1_1.add(GRU(units=50, return_sequences=True, activation='tanh'))
regressorGRU_step_1_1.add(Dropout(0.2))
regressorGRU_step_1_1.add(GRU(units=50, activation='tanh'))
regressorGRU_step_1_1.add(Dropout(0.2))
regressorGRU_step_1_1.add(Dense(units=1))
regressorGRU_step_1_1.compile(optimizer=SGD(learning_rate=0.01, decay=1e-7, momentum=0.9, nesterov=False), loss='mean_squared_error', metrics=['accuracy'])
regressorGRU_step_1_1.fit(x, y, batch_size=150)
y_pred = regressorGRU_step_1_1.predict(x)

# Compare the Best Accurcay Results Of Step-1 and Step -2

# Decision Tree Classifier

In [ ]:
# Accuracy data for Decision Tree step 1
dt_accuracy_step_1 = [0.800000 , 0.800000 , 0.866667 , 0.900000, 0.840000 , 0.800000]

# Accuracy data for Random Forest  Step 2
dt_accuracy_step_2 = [0.960000 , 0.900000 , 0.885714 , 0.900000 , 0.911111 , 0.920000]

# X-axis labels
x_labels = ['5msg', '10msg', '15msg', '20msg', '25msg', '30msg']

# Set the width of each bar
bar_width = 0.35

# Positions of the left bar-boundaries
left_positions = np.arange(len(x_labels))

# Create a figure
fig, ax = plt.subplots()

# Plot the bars for Decision Tree and Random Forest
dt_bars = ax.bar(left_positions, dt_accuracy_step_1, width=bar_width, label='Step_1')
rf_bars = ax.bar(left_positions + bar_width, dt_accuracy_step_2, width=bar_width, label='Step_2')

# Add labels, title, and legend to the plot
ax.set_xlabel('Accuracy')
ax.set_ylabel('Message')
ax.set_title('Accuracy Comparison')
ax.set_xticks(left_positions + bar_width/2)
ax.set_xticklabels(x_labels)
ax.legend()

# Show the plot
plt.show()

# Random Forest Classification

In [ ]:
# Accuracy data for Decision Tree Step 1
dt_accuracy_step_1 = [1.000000 , 1.000000 , 1.000000 , 1.000000 , 0.920000 , 0.900000]

# Accuracy data for Decision Tree Step 2
dt_accuracy_step_2 = [0.960000 , 0.900000 , 0.885714 , 0.900000 , 0.911111 , 0.920000]

# X-axis labels
x_labels = ['5msg', '10msg', '15msg', '20msg', '25msg', '30msg']

# Set the width of each bar
bar_width = 0.35

# Positions of the left bar-boundaries
left_positions = np.arange(len(x_labels))

# Create a figure
fig, ax = plt.subplots()

# Plot the bars for Decision Tree and Random Forest
dt_bars = ax.bar(left_positions, dt_accuracy_step_1, width=bar_width, label='Step_1')
rf_bars = ax.bar(left_positions + bar_width, dt_accuracy_step_2, width=bar_width, label='Step_2')

# Add labels, title, and legend to the plot
ax.set_xlabel('Accuracy')
ax.set_ylabel('Message')
ax.set_title('Accuracy Comparison')
ax.set_xticks(left_positions + bar_width/2)
ax.set_xticklabels(x_labels)
ax.legend()

# Show the plot
plt.show()

# KNN

In [ ]:
# Accuracy data for KNN Step 1
KNN_accuracy_step_1 = [1.000000 , 1.000000 , 1.000000 , 1.000000 , 0.920000 , 0.900000]

# Accuracy data for KNN Step 2
KNN_accuracy_step_2 = [0.960000 , 0.900000 , 0.885714 , 0.900000 , 0.911111 , 0.920000]

# X-axis labels
x_labels = ['5msg', '10msg', '15msg', '20msg', '25msg', '30msg']

# Set the width of each bar
bar_width = 0.35

# Positions of the left bar-boundaries
left_positions = np.arange(len(x_labels))

# Create a figure
fig, ax = plt.subplots()

# Plot the bars for Decision Tree and Random Forest
dt_bars = ax.bar(left_positions, dt_accuracy_step_1, width=bar_width, label='Step_1')
rf_bars = ax.bar(left_positions + bar_width, dt_accuracy_step_2, width=bar_width, label='Step_2')

# Add labels, title, and legend to the plot
ax.set_xlabel('Accuracy')
ax.set_ylabel('Message')
ax.set_title('Accuracy Comparison')
ax.set_xticks(left_positions + bar_width/2)
ax.set_xticklabels(x_labels)
ax.legend()

# Show the plot
plt.show()

# Evaluation 

## Step 1

# **DTC**

In [ ]:
EVALU_1_dataframe_step_1

# KNN

In [ ]:
EVALU_2_dataframe_step_1

# Random Forest 

In [ ]:
EVALU_3_dataframe_step_1

### Step 2

In [ ]:
def test_cases(model_step_1, model_step_2, dfs):
    model_accuracy = []
    Final_time=[]
    precision_scores=[]
    recall_scores=[]
    f1_scores=[]
    for test in dfs:
        # splitting features and target
        X = test.drop(columns=[3], axis=1)
        y_attacks = test[3]
        
        start_time=time.time()
        y_pred_step_2 = model_step_2.predict(X) 
        End_time=time.time()
        accuracy = accuracy_score(y_attacks, y_pred_step_2)
        precision = precision_score(y_attacks, y_pred_step_2, average='micro')
        recall = recall_score(y_attacks, y_pred_step_2, average='micro')
        f1 = f1_score(y_attacks, y_pred_step_2, average='micro')
        
        final_time=End_time-start_time
               
        Final_time.append(final_time)
        model_accuracy.append(accuracy)
        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)
        
    return model_accuracy, Final_time, precision_scores, recall_scores, f1_scores


# DTC

In [ ]:
accuracy_model

In [ ]:
accuracy_model=test_cases(dcc_model_step_1 , dcc_model_step_2_1 , dfs_attacks)
# display in table
preci_test_step_2_1={ '25msg' : accuracy_model[2][0] , '30msg' : accuracy_model[2][1] , '35msg' : accuracy_model[2][2], '40msg' : accuracy_model[2][3] , '45msg' : accuracy_model[2][4] , '50msg' : accuracy_model[2][5]}
recall_step_2_1={ '25msg' : accuracy_model[3][0] , '30msg' : accuracy_model[3][1] , '35msg' : accuracy_model[3][2], '40msg' : accuracy_model[3][3] , '45msg' : accuracy_model[3][4] , '50msg' : accuracy_model[3][5]}
f1_step_2_1={ '25msg' : accuracy_model[4][0] , '30msg' : accuracy_model[4][1] , '35msg' : accuracy_model[4][2], '40msg' : accuracy_model[4][3] , '45msg' : accuracy_model[4][4] , '50msg' : accuracy_model[4][5]}


prf_dataframe_step_2=pd.DataFrame([preci_test_step_2_1 , recall_step_2_1 ,f1_step_2_1 ] , index=['Preciouse' , 'Recall' , 'F1_score'])
print(prf_dataframe_step_2)


# KNN

In [ ]:
accuracy_model=test_cases(KNN_model_step_1 , KNN_model_step_2_1 , dfs_attacks)
# display in table
preci_test_step_2_2={ '25msg' : accuracy_model[2][0] , '30msg' : accuracy_model[2][1] , '35msg' : accuracy_model[2][2], '40msg' : accuracy_model[2][3] , '45msg' : accuracy_model[2][4] , '50msg' : accuracy_model[2][5]}
recall_step_2_2={ '25msg' : accuracy_model[3][0] , '30msg' : accuracy_model[3][1] , '35msg' : accuracy_model[3][2], '40msg' : accuracy_model[3][3] , '45msg' : accuracy_model[3][4] , '50msg' : accuracy_model[3][5]}
f1_step_2_2={ '25msg' : accuracy_model[4][0] , '30msg' : accuracy_model[4][1] , '35msg' : accuracy_model[4][2], '40msg' : accuracy_model[4][3] , '45msg' : accuracy_model[4][4] , '50msg' : accuracy_model[4][5]}


prf_dataframe_step_2=pd.DataFrame([preci_test_step_2_2 , recall_step_2_2 ,f1_step_2_2 ] , index=['Preciose' , 'Recall' , 'F1_Score'])
print(prf_dataframe_step_2)

# Random Forest

In [ ]:
accuracy_model=test_cases(rf_model_step_1 , rf_model_step_2_1 , dfs_attacks)
# display in table
preci_test_step_2_3={ '25msg' : accuracy_model[2][0] , '30msg' : accuracy_model[2][1] , '35msg' : accuracy_model[2][2], '40msg' : accuracy_model[2][3] , '45msg' : accuracy_model[2][4] , '50msg' : accuracy_model[2][5]}
recall_step_2_3={ '25msg' : accuracy_model[3][0] , '30msg' : accuracy_model[3][1] , '35msg' : accuracy_model[3][2], '40msg' : accuracy_model[3][3] , '45msg' : accuracy_model[3][4] , '50msg' : accuracy_model[3][5]}
f1_step_2_3={ '25msg' : accuracy_model[4][0] , '30msg' : accuracy_model[4][1] , '35msg' : accuracy_model[4][2], '40msg' : accuracy_model[4][3] , '45msg' : accuracy_model[4][4] , '50msg' : accuracy_model[4][5]}


prf_dataframe_step_2_3=pd.DataFrame([preci_test_step_2_3 , recall_step_2_3 ,f1_step_2_3 ] , index=['Preciose' , 'Recall' , 'F1_Score'])
print(prf_dataframe_step_2_3)